# Inspired from tutorial at https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b

Degrade tf version to be able to use object_detection


In [ ]:
%tensorflow_version 1.x

Check that GPU is enabled


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Move to training repository (check that google drive is installed) 

In [ ]:
%cd '/content/drive/My Drive/training_demo/'


Clone the tf models repository





In [ ]:
!git clone https://github.com/tensorflow/models.git


Install packages and dependencies

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Compile the model definition

In [ ]:
%cd /content/drive/My Drive/training_demo/models/research/
!protoc object_detection/protos/*.proto --python_out=.

Set the environment path

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/training_demo/models/research/:/content/drive/My Drive/training_demo/models/research/slim'

Setup object_detection

In [ ]:
!python setup.py build
!python setup.py install

Compute remaining time for the session

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Test installation

In [ ]:
%cd /content/drive/My Drive/training_demo/models/research/object_detection/builders/
!python model_builder_test.py

Partition data set into test and train (jpg+xml.files need to be in folder 'images')

In [ ]:
%cd /content/drive/My Drive/training_demo/preprocessing

#10 percent test data
!python partition_dataset.py -x -i /content/drive/My\ Drive/training_demo/images/igluna2 -r 0.1   

Create csv-files 

In [ ]:
%cd /content/drive/My Drive/training_demo/preprocessing
!python xml_to_csv.py -i /content/drive/My\ Drive/training_demo/images/igluna2/train -o /content/drive/My\ Drive/training_demo/annotations/train_labels.csv
!python xml_to_csv.py -i /content/drive/My\ Drive/training_demo/images/igluna2/test -o /content/drive/My\ Drive/training_demo/annotations/test_labels.csv


Generate TF-records

In [ ]:
%cd /content/drive/My Drive/training_demo/preprocessing

#create train data:
!python generate_tfrecord.py --label=potato --csv_input=/content/drive/My\ Drive/training_demo/annotations/train_labels.csv --img_path=/content/drive/My\ Drive/training_demo/images/igluna2/train --output_path=/content/drive/My\ Drive/training_demo/annotations/train.record

# Create test data:
!python generate_tfrecord.py --label=potato --csv_input=/content/drive/My\ Drive/training_demo/annotations/test_labels.csv --img_path=/content/drive/My\ Drive/training_demo/images/igluna2/test --output_path=/content/drive/My\ Drive/training_demo/annotations/test.record


Get pre-trained model

In [ ]:
%cd /content/drive/My Drive/training_demo/pre-trained-models


!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
!tar -xvf faster_rcnn_resnet101_coco_2018_01_28.tar.gz

Background tracking checkpoints (not necessary)

In [ ]:
%cd /content/drive/My Drive/training_demo
%load_ext tensorboard
%tensorboard --logdir training_v2/

ACTUAL TRAINING


In [ ]:

%cd /content/drive/My Drive/training_demo
!python train.py --alsologtostderr --train_dir=training/ --pipeline_config_path=config_files/faster_rcnn_resnet101.config


Export TRained Model

In [ ]:
%cd /content/drive/My Drive/training_demo

#change checkpoint prefix
!python models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path config_files/ssd_mobilenet_v1_coco.config --trained_checkpoint_prefix training/model.ckpt-18577 --output_directory saved_trained_models/mobilenet_v1_2